In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    api_key="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"
)


response = llm.invoke("What is the full name of Sachin T")
print(response.content)


The full name of Sachin T is Sachin Tendulkar. He is a former Indian international cricketer and widely regarded as one of the greatest batsmen in the history of cricket.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/content/en/locations/india/bangalore")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.
















Bangalore, India
Amazon JobsTeamsLocationsJob categoriesResourcesAccommodationsBenefitsInclusive experiencesInterviewing at AmazonLeadership PrinciplesWorking at AmazonFAQLocaleLanguage: en-USAmazon JobsSearchSearchLocaleLanguage: en-USÂ Search for jobsTitle or keywordLocationSearchSearch Bangalore, IndiaAt Amazon, weâ€™re looking for passionate innovators across the world to create the next best thing for our customers. Explore open roles at each of our locations and apply today.Equal opportunityAmazon is an equal opportunity employer and does not discriminate on the basis of protected veteran status, disability, or other legally protected status.Career areasTeamsLocationsJob categoriesHourly jobsMilitary hiringUniversity talentResourcesAccommodationsBenefitsInclusive experiencesInterviewing at AmazonLeadership PrinciplesWorking at AmazonFAQStart your Amazon journeyShare your resume and basic info to get personalized job recommendations and more.Create an accountAmazon s

In [3]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED DATA FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped data is from the career's page of a website
    Your job is to extract the job posting and return them in JSON format containing the 
    following keys: "role","experience","skills" and "description".
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
    
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
[
  {
    "role": "Software Development Engineer",
    "experience": "2+ years",
    "skills": ["Java", "Python", "C++"],
    "description": "Design, develop, and test software applications"
  },
  {
    "role": "Data Scientist",
    "experience": "5+ years",
    "skills": ["Machine Learning", "Data Analysis", "Statistics"],
    "description": "Analyze and interpret complex data to inform business decisions"
  },
  {
    "role": "Product Manager",
    "experience": "3+ years",
    "skills": ["Product Development", "Marketing", "Leadership"],
    "description": "Develop and launch new products to drive business growth"
  }
]
```


In [24]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)
type(json_res)


[{'role': 'Software Development Engineer', 'experience': '2+ years', 'skills': ['Java', 'Python', 'C++'], 'description': 'Design, develop, and test software applications'}, {'role': 'Data Scientist', 'experience': '5+ years', 'skills': ['Machine Learning', 'Data Analysis', 'Statistics'], 'description': 'Analyze and interpret complex data to inform business decisions'}, {'role': 'Product Manager', 'experience': '3+ years', 'skills': ['Product Development', 'Marketing', 'Leadership'], 'description': 'Develop and launch new products to drive business growth'}]


list

In [11]:
import pandas as pd

df = pd.read_csv("techstack_links.csv")
print(df)

                                  Techstack  \
0                   React, Node.js, MongoDB   
1                  Angular,.NET, SQL Server   
2         Vue.js, Ruby on Rails, PostgreSQL   
3                     Python, Django, MySQL   
4               « Java, Spring Boot, Oracle   
5              7 Flutter, Firebase, GraphQL   
6                     WordPress, PHP, MySQL   
7                       Magento, PHP, MySQL   
8            React Native, Node.js, MongoDB   
9                     iOS, Swift, Core Data   
10          Android, Java, Room Persistence   
11                Kotlin, Android, Firebase   
12          Android TV, Kotlin, Android NDK   
13                        IOS, Swift, ARKit   
14           Cross-platform, Xamarin, Azure   
15            Backend, Kotlin, Spring Boot    
16            Frontend, TypeScript, Angular   
17       Full-stack, JavaScript, Express.js   
18   Machine Learning, Python, TensorFlow _   
19                  DevOps, Jenkins, Docker   

            

In [12]:
import uuid
import chromadb
from chromadb import PersistentClient
client = PersistentClient(path="vectorstore")


client = chromadb.PersistentClient(path='vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas=[{"links": row["Links"]}],
            ids=[str(uuid.uuid4())]
        )


In [26]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas')
print(links)

[[{'links': 'https://example.com/java-portfolio'}, {'links': 'https://example.com/android-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}]]


In [27]:
print(job)

{'role': 'Software Development Engineer', 'experience': '2+ years', 'skills': ['Java', 'Python', 'C++'], 'description': 'Design, develop, and test software applications'}


In [25]:
job = json_res[0]
print(job['skills'])

['Java', 'Python', 'C++']


In [31]:
prompt_email = PromptTemplate.from_template(
    """
    #JOB DESCRIPTION:
    {job_description}
    
    #INSTRUCTION:
    You are Prasanna,a business development executive at Amazon.Amazon is an Product based company dedicated
    the seamless intergration of business processes through automated tools.
    Your job is to write a cold email to the client regarding the job mentioned above in fulfilling
    thier needs.
    Also add the most relevant ones from the following links to showcase the company's portfolio: {link_list}
    Remember you are Prasanna , BDE at Amazon , Banglore
    Make it more in a professional format
    Mention as writing email to Hiring Manager also mention my email : iamprasanna1@gmail.com
    Do not provide a preamble
    ### EMAIL (NO PREAMBLE):
    """ 
)
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Development Engineer for Seamless Business Process Integration

Dear Hiring Manager,

I am Prasanna, a Business Development Executive at Amazon, Bangalore. I came across your job posting for a Software Development Engineer and was impressed by the role's requirements. As a leading product-based company dedicated to the seamless integration of business processes through automated tools, I believe Amazon can fulfill your needs.

The job description mentions designing, developing, and testing software applications, which aligns with our expertise in Java, Python, and C++. With over 2+ years of experience in software development, our team can deliver high-quality solutions that meet your expectations.

I would like to highlight some of our notable portfolios that demonstrate our capabilities:

* Java expertise: https://example.com/java-portfolio
* Python expertise: https://example.com/python-portfolio and https://example.com/ml-python-portfolio
* Android portfolio: